# Focal cell coordinate finder

In [1]:
import calculate_radial_analysis as calculate
import dataio

import glob
import os
import re
from tqdm.auto import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## 1) Event dictionary loader

Specify cell type to filter out unwanted apoptoses, want end result of event_dict that takes the format of :`event_dict = {event_ID:event_time}` 

where `event_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}_{cell_ID}_{cell_type}` 

For example: 

```
event_dict = {'GV0794_Pos4_393_GFP': '766',
             'GV0794_Pos7_364_GFP': '550',
             'GV0794_Pos7_457_GFP': '805',
             'GV0794_Pos7_369_GFP': '833',
             'GV0794_Pos7_292_GFP': '884',
             'GV0795_Pos4_358_GFP': '628',
             'GV0795_Pos4_18_GFP': '855',
             'GV0795_Pos4_1356_GFP': '992',
             'GV0795_Pos4_359_GFP': '1133',
             'GV0796_Pos4_542_GFP': '581',
             'GV0796_Pos4_1334_GFP': '719',
             'GV0796_Pos4_1586_GFP': '785'}
 ```
 
Function to apoptosis event dictionary from preexisting data store:

`dataio.apoptosis_list_loader(path_to_apop_lists, cell_type)`

Where the `path_to_apop_lists` is the directory containing the apoptosis information and the `cell_type` is either `'Scr'` or `'WT'`.

In [2]:
import json
f = open('/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/control_wt_events_original.json')
event_dict = json.load(f)
event_dict

{'GV0794_Pos4_4_GFP_23': 23,
 'GV0794_Pos4_5_GFP_132': 132,
 'GV0794_Pos4_7_GFP_24': 24,
 'GV0794_Pos4_10_GFP_66': 66,
 'GV0794_Pos4_12_GFP_800': 800,
 'GV0794_Pos4_14_GFP_115': 115,
 'GV0794_Pos4_14_GFP_156': 156,
 'GV0794_Pos4_14_GFP_158': 158,
 'GV0794_Pos4_19_GFP_38': 38,
 'GV0794_Pos4_20_GFP_84': 84,
 'GV0794_Pos4_20_GFP_236': 236,
 'GV0794_Pos4_21_GFP_58': 58,
 'GV0794_Pos4_22_GFP_125': 125,
 'GV0794_Pos4_44_GFP_79': 79,
 'GV0794_Pos4_65_GFP_170': 170,
 'GV0794_Pos4_65_GFP_281': 281,
 'GV0794_Pos4_101_GFP_505': 505,
 'GV0794_Pos4_102_GFP_403': 403,
 'GV0794_Pos4_108_GFP_653': 653,
 'GV0794_Pos4_113_GFP_402': 402,
 'GV0794_Pos4_122_GFP_338': 338,
 'GV0794_Pos4_123_GFP_428': 428,
 'GV0794_Pos4_123_GFP_571': 571,
 'GV0794_Pos4_125_GFP_340': 340,
 'GV0794_Pos4_133_GFP_461': 461,
 'GV0794_Pos4_135_GFP_489': 489,
 'GV0794_Pos4_140_GFP_383': 383,
 'GV0794_Pos4_147_GFP_376': 376,
 'GV0794_Pos4_148_GFP_399': 399,
 'GV0794_Pos4_149_GFP_447': 447,
 'GV0794_Pos4_151_GFP_562': 562,
 'GV0794_P

In [3]:
len(event_dict)

10491

In [6]:
f = open('/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/control_wt_events.json')
event_dict = json.load(f)
event_dict

{'GV0794_Pos4_4_GFP': 23,
 'GV0794_Pos4_5_GFP': 132,
 'GV0794_Pos4_7_GFP': 24,
 'GV0794_Pos4_10_GFP': 66,
 'GV0794_Pos4_12_GFP': 800,
 'GV0794_Pos4_14_GFP': 158,
 'GV0794_Pos4_19_GFP': 38,
 'GV0794_Pos4_20_GFP': 236,
 'GV0794_Pos4_21_GFP': 58,
 'GV0794_Pos4_22_GFP': 125,
 'GV0794_Pos4_44_GFP': 79,
 'GV0794_Pos4_65_GFP': 281,
 'GV0794_Pos4_101_GFP': 505,
 'GV0794_Pos4_102_GFP': 403,
 'GV0794_Pos4_108_GFP': 653,
 'GV0794_Pos4_113_GFP': 402,
 'GV0794_Pos4_122_GFP': 338,
 'GV0794_Pos4_123_GFP': 571,
 'GV0794_Pos4_125_GFP': 340,
 'GV0794_Pos4_133_GFP': 461,
 'GV0794_Pos4_135_GFP': 489,
 'GV0794_Pos4_140_GFP': 383,
 'GV0794_Pos4_147_GFP': 376,
 'GV0794_Pos4_148_GFP': 399,
 'GV0794_Pos4_149_GFP': 447,
 'GV0794_Pos4_151_GFP': 562,
 'GV0794_Pos4_153_GFP': 753,
 'GV0794_Pos4_157_GFP': 400,
 'GV0794_Pos4_167_GFP': 455,
 'GV0794_Pos4_168_GFP': 639,
 'GV0794_Pos4_169_GFP': 697,
 'GV0794_Pos4_170_GFP': 460,
 'GV0794_Pos4_172_GFP': 522,
 'GV0794_Pos4_177_GFP': 604,
 'GV0794_Pos4_178_GFP': 618,
 'GV07

In [9]:
len(event_dict)

8901

## 2) Load corresponding cell tracking data and specifiy other parameters

`dataio.hdf5_file_finder(parent_folder_hdf5)` loads a list of all hdf5 files when supplied with the parent directory of all the experiments. This relies on a typical sub-parent directory structure of `'GV****/Pos*/*aligned/HDF/segmented.hdf5'` (aligned folder optional)

In [4]:
### load tracking data
parent_folder_hdf5 = '/home/nathan/data/kraken/scr/h2b/giulia'
hdf5_file_list = dataio.hdf5_file_finder(parent_folder_hdf5)


In [12]:


hdf5_file_path, error_log, success_log = [], [], []
 
for apop_ID in tqdm(event_dict):

    expt = "GV" + str(re.findall(r"GV(\d+)", apop_ID)[0])
    position = re.findall(r"Pos(\d+)", apop_ID)[0]

    position = "Pos" + position

    expt_position = os.path.join(
        expt, position, ""
    )  ## additional '' here so that / added to end of string

    cell_ID = int((re.findall(r"(\d+)_.FP", apop_ID))[0])
    print("ID", apop_ID, "experiment:", expt_position)

    # checks if hdf5 file path already loaded to avoid repeat loading
    if expt_position not in hdf5_file_path:
        ## load that track data
        print("Loading", expt_position)
        hdf5_file_path = [
            hdf5_file_path
            for hdf5_file_path in hdf5_file_list
            if expt_position in hdf5_file_path
        ][0]
        wt_cells, scr_cells, all_cells = dataio.load_tracking_data(hdf5_file_path)
        print("Loaded", expt_position)

    if "RFP" in apop_ID:
        cell_ID = -cell_ID

    focal_time = int(event_dict[apop_ID])

    target_cell = [cell for cell in all_cells if cell.ID == cell_ID][0]

    focal_cell = pd.DataFrame(columns = ['x', 'y', 't', 'ID', 'state'])

    focal_cell['x'], focal_cell['y'],focal_cell['t'], focal_cell['ID'], focal_cell['state'] =target_cell['x'], target_cell['y'],target_cell['t'], target_cell['ID'], target_cell['state']  

    fn = f'/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/control_event_tracks/{apop_ID}.csv'

    focal_cell.to_csv(fn)

  0%|          | 0/8901 [00:00<?, ?it/s]

[INFO][2022/03/09 05:10:35 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 05:10:35 PM] Loading tracks/obj_type_1


ID GV0794_Pos4_4_GFP experiment: GV0794/Pos4/
Loading GV0794/Pos4/
Btrack version no.: 0.4.3


[INFO][2022/03/09 05:10:37 PM] Loading objects/obj_type_1 (69360, 5) (64986 filtered: area>=100)
[INFO][2022/03/09 05:10:39 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 05:10:39 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 05:10:39 PM] Loading tracks/obj_type_2
[INFO][2022/03/09 05:10:39 PM] Loading objects/obj_type_2 (13865, 5) (10538 filtered: area>=100)
[INFO][2022/03/09 05:10:40 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0794/Pos4/
ID GV0794_Pos4_5_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_7_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_10_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_12_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_14_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_19_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_20_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_21_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_22_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_44_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_65_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_101_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_102_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_108_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_113_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_122_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_123_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_125_GFP experiment: GV0794/Pos4/
ID GV0794_Pos4_133_GFP experiment: GV0794/Pos4/


...

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0807/Pos4/
ID GV0807_Pos4_23_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_98_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_112_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_143_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_147_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_150_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_167_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_179_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_191_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_199_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_202_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_225_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_250_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_255_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_272_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_276_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_316_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_328_GFP experiment: GV0807/Pos4/
ID GV0807_Pos4_352_GFP experiment: GV

[INFO][2022/03/09 05:19:30 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0807/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 05:19:30 PM] Loading tracks/obj_type_1


ID GV0807_Pos5_9_GFP experiment: GV0807/Pos5/
Loading GV0807/Pos5/
Btrack version no.: 0.4.3


[INFO][2022/03/09 05:19:31 PM] Loading objects/obj_type_1 (129954, 5) (111411 filtered: area>=100)
[INFO][2022/03/09 05:19:33 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0807/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 05:19:33 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0807/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 05:19:33 PM] Loading tracks/obj_type_2
[INFO][2022/03/09 05:19:34 PM] Loading objects/obj_type_2 (87213, 5) (74295 filtered: area>=100)
[INFO][2022/03/09 05:19:34 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0807/Pos5/Pos5_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0807/Pos5/
ID GV0807_Pos5_32_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_35_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_51_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_143_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_171_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_203_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_209_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_224_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_226_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_235_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_256_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_270_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_272_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_277_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_288_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_315_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_316_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_317_GFP experiment: GV0807/Pos5/
ID GV0807_Pos5_320_GFP experiment: GV0

In [43]:
### finding all scr apoptoses in uninduced datasets?